In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
os.chdir('..')
os.getcwd()

'/home/yermaand/audio-nn'

In [3]:
from sklearn.model_selection import KFold
import numpy as np
import yaml

In [10]:
dataset_name = '002_eyedea_RX100_48'
seed_cross_validation = 42
seed_permutation = 48

os.makedirs(f'config/training_files/{dataset_name}')
os.makedirs(f'config/validation_files/{dataset_name}')
os.makedirs(f'config/testing_files/{dataset_name}')

In [11]:
dataset = f'config/dataset/{dataset_name}.yaml'

with open(dataset, 'r') as stream:
    files = yaml.safe_load(stream)

files = [f'- {f}' for f in files]
files = np.array(files)

folds = KFold(n_splits=5, random_state=seed_cross_validation, shuffle=True).split(files)

print(int(len(files) * 0.6), int(len(files) * 0.2), int(len(files) * 0.2))
print('-' * 10)

np.random.seed(seed_permutation)

for i, (trn_indices, tst_indices) in enumerate(folds):
    trn_files = files[trn_indices]
    tst_files = files[tst_indices]

    trn_files = np.random.permutation(trn_files)

    p = int(len(trn_files) * 0.75)
    trn_files, val_files = trn_files[:p], trn_files[p:]
    
    trn_files = sorted(trn_files)
    val_files = sorted(val_files)

    print(len(trn_files), len(val_files), len(tst_files))

    np.savetxt(f'config/training_files/{dataset_name}/{i}.yaml', trn_files, fmt='%s')
    np.savetxt(f'config/validation_files/{dataset_name}/{i}.yaml', val_files, fmt='%s')
    np.savetxt(f'config/testing_files/{dataset_name}/{i}.yaml', tst_files, fmt='%s')

21 7 7
----------
21 7 8
21 8 7
21 8 7
21 8 7
21 8 7


In [9]:
# trn_files